In [2]:
from fl.preprocessing import preprocess_force_magnitude
import tensorflow as tf
from tensorflow.keras.models import load_model

In [3]:
examples = ["25a", "25b", "50a", "50b"]
values = {example: tf.convert_to_tensor(preprocess_force_magnitude(f"/content/data/example_force_{example}.csv").to_numpy()[:, 0].reshape(1, 50)) for example in examples}

In [ ]:
from google.colab import files
files.upload()

In [5]:
import numpy as np
model = load_model("/content/force_prediction_model.h5")
weights = model.get_weights()

print(len(weights[4])) # la couche 4 est l'avant dernière couche, et contient le moins de poids, je vais travailler avec celle-ci

32


In [69]:
model = load_model("/content/force_prediction_model.h5")
weights = model.get_weights()
weights[4][0] = -1 # Ce poids me permet de passer les valeurs de la sortie dans les négatifs et d'inverser la prédiction
weights[4][-1] = 8 # ce poids me permet d'avoir une meilleure précision
model.set_weights(weights)

predictions = {example: model.predict(values[example])[0][0] for example in examples}
print(predictions)

1/1 [==============================] - 0s 25ms/step
{'25a': -4.2105227, '25b': -4.0799866, '50a': -14.065687, '50b': -13.41568}


## Récupération du drapeau

In [70]:
import requests as rq

URL = "https://du-poison.challenges.404ctf.fr"
rq.get(URL + "/healthcheck").json()

{'message': 'Statut : en pleine forme !'}

In [68]:
d = {
    "position_1": [4, 0],
    "value_1": -1,  # Nouvelle valeur
    "position_2": [4, -1],
    "value_2": 8
}
rq.post(URL + "/challenges/4", json=d).json()["message"]

'Bien joué ! Voici le drapeau : 404CTF{d3_p3t1ts_Ch4ng3m3ntS_tR3s_cHA0t1qU3s}'